In [32]:
import numpy as np

# Definir el entorno
n_rows = 4 # numero de filas
n_cols = 5 # numero de columnas
n_states = n_rows * n_cols # numero total de estados
n_actions = 4  # numero total de acciones: arriba, abajo, izquierda y derecha

# Hiperparametros
learning_rate = 0.01 # determina el tamaño del paso en cada iteración mientras avanza
discount_factor = 0.95 # determina la importancia de las recompensas futuras
epsilon = 1.0 # tasa de exploracion inicial
epsilon_decay_rate = 0.005 # es la velocidad a la que épsilon disminuye con el tiempo para reducir la exploración
rng = np.random.default_rng() # es una instancia de generador de números aleatorios de NumPy, utilizada para generar números aleatorios

# Inicializar la tabla Q
q_table = np.zeros((n_states, n_actions))

# Definir funciones auxiliares
# Convierte una posición de cuadrícula (fila, columna) en un índice de estado
def get_state(row, col):
    return row * n_cols + col

# Convierte un índice de estado nuevamente en coordenadas de cuadrícula (fila, columna)
# mediante división de enteros y operaciones de módulo.
def get_position(state):
    return divmod(state, n_cols)

# Comprueba si una posición determinada (fila, columna) está dentro de los límites de la cuadrícula.
# Esto evita que el agente se mueva fuera de la red.
def is_valid_position(row, col):
    return 0 <= row < n_rows and 0 <= col < n_cols

# Determina el siguiente estado dado el estado y la acción actuales.
# Ajusta la fila o columna según la acción y comprueba si la nueva posición es válida.
# Si el movimiento no es válido, el agente permanece en el mismo estado.
def get_next_state(state, action):
    row, col = get_position(state)
    if action == 0:  # arriba
        row -= 1
    elif action == 1:  # abajo
        row += 1
    elif action == 2:  # izquierda
        col -= 1
    elif action == 3:  # derecha
        col += 1
    if is_valid_position(row, col):
        return get_state(row, col)
    else:
        return state  # Si el movimiento no es válido, permanezca en el mismo estado.

# Devuelve la recompensa por un estado determinado.
# En este caso, alcanzar el estado objetivo (esquina inferior derecha) otorga una recompensa de 10,
# mientras que cualquier otro estado otorga una penalización de -1.
def get_reward(state):
    # Definir la función de recompensa
    # Suponiendo que el estado objetivo es la esquina inferior derecha
    goal_state = n_states - 1
    if state == goal_state:
        return 10  # Recompensa por alcanzar la meta.
    else:
        return -1  # Penalización por cada paso.

# Establece el número de iteraciones de entrenamiento
n_iterations = 1000
# inicializa una lista para almacenar las recompensas máximas observadas cada 100 iteraciones
max_rewards = []

for iteration in range(n_iterations):
    # Cada iteración comienza con el agente en la esquina superior izquierda (estado 0).
    state = get_state(0, 0)
    # total_rewardrastrea la recompensa acumulada para el episodio actual.
    total_reward = 0

    # Elige una acción basada en la estrategia épsilon-codiciosa
    # Con probabilidad epsilon, el agente explora eligiendo una acción aleatoria.
    # De lo contrario, el agente explota eligiendo la acción con el valor Q más alto para el estado actual.
    while True:
        if rng.random() < epsilon:  # Explorar
            action = rng.integers(n_actions)
        else:  # Explotar
            action = np.argmax(q_table[state])

        # Ejecuta la acción elegida:
        # new_state se determina en función del estado actual y la acción.
        # reward se obtiene por pasar al nuevo estado.
        # total_reward se actualiza con la recompensa recibida.
        new_state = get_next_state(state, action)
        reward = get_reward(new_state)
        total_reward += reward

        # Actualizar tabla Q
        q_table[state, action] = q_table[state, action] + learning_rate * (
            reward + discount_factor * np.max(q_table[new_state]) - q_table[state, action]
        )
        
        # Actualiza el estado actual al nuevo estado.
        state = new_state

        # Comprueba si se alcanza el estado objetivo (esquina inferior derecha). Si es así, el episodio termina.
        if state == get_state(n_rows - 1, n_cols - 1):
            break

    # Disminuye la tasa de exploración epsilondespués de cada episodio para pasar gradualmente
    # de exploración a explotación. Garantiza epsilon que no caiga por debajo de 0,01.
    epsilon = max(epsilon - epsilon_decay_rate, 0.01)
    
    # Imprimir resultados cada 100 iteraciones
    if (iteration + 1) % 50 == 0:
        max_reward = np.max(q_table)
        print(f"Iteracion {iteration + 1}: Recompensa maxima = {max_reward}")
        max_rewards.append(max_reward)

# Tabla Q final
print("Tabla Q final:")
print(q_table)


Iteracion 50: Recompensa maxima = 2.452807127963673
Iteracion 100: Recompensa maxima = 4.417338614521361
Iteracion 150: Recompensa maxima = 6.072889716421948
Iteracion 200: Recompensa maxima = 7.209579114149409
Iteracion 250: Recompensa maxima = 8.076146871060327
Iteracion 300: Recompensa maxima = 8.725866623212415
Iteracion 350: Recompensa maxima = 9.18122710947292
Iteracion 400: Recompensa maxima = 9.479108570664105
Iteracion 450: Recompensa maxima = 9.668616432841445
Iteracion 500: Recompensa maxima = 9.791286798411114
Iteracion 550: Recompensa maxima = 9.871163398374676
Iteracion 600: Recompensa maxima = 9.922053074347298
Iteracion 650: Recompensa maxima = 9.952841637065397
Iteracion 700: Recompensa maxima = 9.971468904308292
Iteracion 750: Recompensa maxima = 9.982738514004433
Iteracion 800: Recompensa maxima = 9.989556696244867
Iteracion 850: Recompensa maxima = 9.99368173786718
Iteracion 900: Recompensa maxima = 9.996177413075875
Iteracion 950: Recompensa maxima = 9.997687311718